# SME0823 Modelos de Regressão e Aprendizado Supervisionado 2

Victor Andreas Iwanaga

Prova 2

---

## Mão na massa

In [ ]:
# pip install --upgrade --force-reinstall pandas numpy

---

## Parte 1.0 - Preparação Inicial (Importação e Leitura)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf

# Carregando os dados corretos para o seu USP (final 9 -> sneeze5)
url = 'https://raw.githubusercontent.com/cibelerusso/Datasets/refs/heads/main/sneeze5.csv'
df = pd.read_csv(url)
df.head()

In [ ]:
if "Unnamed: 0" in df.columns:
    df = df.iloc[:, 1:]

display(df.head())
df.info()

In [ ]:
df.columns

---

## Parte 1.1 - Análise Exploratória

In [ ]:
# Histograma da variável resposta
plt.figure(figsize=(8, 4))
sns.histplot(df['nsneeze'], kde=True, bins=20)
plt.title('Distribuição do Número de Espirros (nsneeze)')
plt.show()


In [ ]:
# Relação com variáveis categóricas
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
sns.boxplot(x='alcohol', y='nsneeze', data=df, ax=axes[0])
sns.boxplot(x='antihist', y='nsneeze', data=df, ax=axes[1])
sns.boxplot(x='smoker', y='nsneeze', data=df, ax=axes[2])
plt.show()

In [ ]:
# Relação com variáveis numéricas (Pollen e Age)
sns.pairplot(df, x_vars=['age', 'pollen'], y_vars='nsneeze', height=4, kind='reg', plot_kws={'line_kws':{'color':'red'}})
plt.show()

In [ ]:
# Estatísticas descritivas
display(df.describe())

A análise exploratória indica que a variável ***nsneeze (Número de espirros)*** resposta é uma contagem assimétrica, sugerindo o uso da família Poisson. As variáveis explicativas parecem ter impacto no desfecho: o pólen e o tabagismo parecem aumentar os espirros, enquanto o uso de anti-histamínico parece reduzi-los.

---

## Parte 2.0 - Ajuste do modelo de Poisson

In [ ]:
import statsmodels.formula.api as smf
import statsmodels.api as sm

In [ ]:
resposta = df['nsneeze']
preditoras = df[['alcohol', 'antihist', 'smoker', 'age', 'pollen']]

formula = "nsneeze ~ C(alcohol) + C(antihist) + C(smoker) + age + pollen"

modelo_pois = smf.glm(
    formula=formula,
    data=df,
    family=sm.families.Poisson()
)

ajuste_pois = modelo_pois.fit()
print(ajuste_pois.summary())

**Interpretação do Modelo de Poisson**:
Com base no sumário do modelo ajustado acima, aqui estão as interpretações:
1. **Estimativa dos Coeficientes**:
Analisando os sinais e valores dos coeficientes estimados (coef), podemos entender o impacto de cada variável no número médio de espirros (nsneeze):

- 'alcohol' (0.3477): O coeficiente é positivo, indicando que o consumo de álcool está associado a um aumento na taxa média de espirros em comparação com quem não consome, mantendo as outras variáveis constantes.
- 'antihist' (-0.5969): O coeficiente é negativo, o que sugere que o uso de anti-histamínicos está associado a uma redução significativa no número esperado de espirros (o que faz sentido, já que é um medicamento para alergia).
- 'smoker' (0.6700): O coeficiente é positivo, mostrando que fumantes tendem a ter uma quantidade de espirros maior do que os não fumantes.
- 'age' (-0.0127): O valor é negativo, indicando uma leve tendência de diminuição no número de espirros conforme a idade do paciente aumenta.
- 'pollen' (0.0301): O coeficiente é positivo, confirmando que quanto maior a concentração de pólen no ar, maior é o número esperado de espirros.

2. **Significância Estatística**:
Ao observar a coluna P>|z| (p-valor), notamos que todas as variáveis apresentaram um valor de 0.000.Isso significa que, considerando um nível de significância padrão de 5% ($\alpha = 0.05$), todos os coeficientes são estatisticamente significativos. Ou seja, temos fortes evidências de que o álcool, o uso de anti-histamínico, o tabagismo, a idade e o nível de pólen influenciam, de fato, o número de espirros diários neste grupo.

---

## Parte 3 - Verificação de indícios de superdispersão no modelo de Poisson
 Verifique se há indícios de superdispersão no modelo de Poisson ajustado no item 2 por, pelo menos, dois métodos diferentes.  

Interprete os resultados e conclua se o modelo de Poisson é adequado em termos de dispersão.


In [ ]:
# Método 1: Cálculo da Estatística de Dispersão (Phi)
# Phi = Qui-quadrado de Pearson / Graus de Liberdade
phi = ajuste_pois.pearson_chi2 / ajuste_pois.df_resid

print(f"Qui-quadrado de Pearson: {ajuste_pois.pearson_chi2:.2f}")
print(f"Graus de Liberdade: {ajuste_pois.df_resid}")
print(f"Estatística de Dispersão (Phi): {phi:.4f}")

# Comparação rápida: Média vs Variância dos dados brutos
print(f"\nMédia da variável resposta (nsneeze): {df['nsneeze'].mean():.2f}")
print(f"Variância da variável resposta (nsneeze): {df['nsneeze'].var():.2f}")

In [ ]:
# Método 2: Gráfico de Envelope Simulado (Código fornecido)
def envelope_poisson(fitted_model, X, title):
    resid_dev = fitted_model.resid_deviance.copy()
    sorted_resid = np.sort(resid_dev)

    sim_resid = []
    for _ in range(100):
        mu_sim = np.clip(fitted_model.fittedvalues, 1e-3, 1e5)
        y_sim = np.random.poisson(mu_sim)
        sim_model = sm.GLM(y_sim, X, family=sm.families.Poisson()).fit()
        sim_resid.append(np.sort(sim_model.resid_deviance))

    sim_resid = np.array(sim_resid)
    lower = np.percentile(sim_resid, 2.5, axis=0)
    upper = np.percentile(sim_resid, 97.5, axis=0)

    plt.plot(sorted_resid, 'o', label="Resíduos observados")
    plt.plot(lower, 'r--', linewidth=1, label="banda 2.5%")
    plt.plot(upper, 'r--', linewidth=1, label="banda 97.5%")
    plt.title(title)
    plt.xlabel("Ordem dos resíduos")
    plt.ylabel("Resíduo componente do desvio")
    plt.legend()
    plt.grid(True)

# Gerando o gráfico
results = ajuste_pois
X = preditoras
plt.figure(figsize=(8, 3), dpi=150)
envelope_poisson(results, X, "Envelope de Resíduos - Poisson")

**Interpretação dos Resultados da Dispersão**
1. Pelo Método Numérico:
Os cálculos mostram que o pressuposto básico da distribuição de Poisson (Média = Variância) foi violado:

- A Variância da variável resposta (146.06) é drasticamente maior que a Média (4.87), sendo quase 30 vezes superior.
- **A Estatística de Dispersão ($\phi$)**, calculada pela razão entre o Qui-quadrado de Pearson e os Graus de Liberdade, resultou em 7.84. Como este valor é muito superior a 1, temos uma evidência numérica clara de superdispersão.


2. Pelo Método Gráfico (Envelope de Resíduos):O gráfico de envelope reforça a conclusão numérica. Observa-se que a grande maioria dos pontos (resíduos observados) ultrapassa o limite da banda de confiança de 97.5% (a linha vermelha superior). Isso indica que o modelo de Poisson não consegue capturar a variabilidade excessiva presente nos dados, resultando em um ajuste pobre.


**Conclusão**:Devido à forte evidência de superdispersão detectada por ambos os métodos, conclui-se que o modelo de Poisson não é adequado para estes dados. O uso deste modelo subestimaria os erros-padrão, levando a conclusões equivocadas sobre a significância das variáveis. Recomenda-se o ajuste de um modelo que flexibilize a relação média-variância, como o modelo Binomial Negativo.

---

## Parte 4 - Ajuste de um modelo  **Binomial Negativo**
Caso seja detectada superdispersão, ajuste um modelo **Binomial Negativo** com a mesma estrutura de regressão do item 2. Compare os ajustes de Poisson e Binomial Negativa por meio de:

* desvio (deviance),
* AIC,
* gráficos de resíduos componentes do desvio.

Discuta qual modelo é mais adequado para descrever o número de espirros, justificando sua resposta com base nas métricas e nos diagnósticos gráficos.

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

formula = "nsneeze ~ C(alcohol) + C(antihist) + C(smoker) + age + pollen"

alpha_fixado = 0.2

#alpha para sneeze1 a sneeze5 = 0.23, 0.3, 0.25,  0.2, 0.2

modelo_nb = smf.glm(
    formula=formula,
    data=df,
    family=sm.families.NegativeBinomial(alpha=alpha_fixado)
)

ajuste_nb = modelo_nb.fit()
print(ajuste_nb.summary())

alpha_usado = ajuste_nb.family.alpha
print("Alpha usado no modelo:", alpha_usado)

alpha_est = ajuste_nb.params.iloc[-1]
print("Alpha estimado:", alpha_est)

Comparar as métricas (AIC e Deviance).

In [ ]:
print("=== Comparação dos Modelos ===")
print(f"Poisson - Deviance: {ajuste_pois.deviance:.2f}")
print(f"Binomial Negativa - Deviance: {ajuste_nb.deviance:.2f}")
print("-" * 30)
print(f"Poisson - AIC: {ajuste_pois.aic:.2f}")
print(f"Binomial Negativa - AIC: {ajuste_nb.aic:.2f}")

# Diferença de AIC (se > 10, a diferença é muito forte a favor do menor)
diff_aic = ajuste_pois.aic - ajuste_nb.aic
print("-" * 30)
print(f"Redução no AIC (Poisson -> BinNeg): {diff_aic:.2f}")

Gráfico de envelope para o novo modelo.

In [ ]:
# Célula 2: Diagnóstico Gráfico - Envelope Simulado (Binomial Negativa)
def envelope_negbin(fitted_model, title):
    # 1. Extrair resíduos e dados
    resid_dev = fitted_model.resid_deviance.copy()
    sorted_resid = np.sort(resid_dev)
    X = fitted_model.model.exog
    mu = np.clip(fitted_model.fittedvalues, 1e-3, 1e5)
    
    # Recupera o alpha do modelo (0.2)
    alpha = fitted_model.family.alpha
    
    # 2. Simulação
    sim_resid = []
    for _ in range(100):
        # Parametrização para o numpy (n, p) baseada na média (mu) e alpha
        # Var = mu + alpha * mu^2
        # n = 1/alpha
        # p = n / (n + mu)
        n_param = 1 / alpha
        p_param = n_param / (n_param + mu)
        
        # Gera resposta simulada Y ~ BinNeg
        y_sim = np.random.negative_binomial(n=n_param, p=p_param)
        
        # Reajusta o modelo
        sim_model = sm.GLM(
            y_sim, X, 
            family=sm.families.NegativeBinomial(alpha=alpha)
        ).fit()
        sim_resid.append(np.sort(sim_model.resid_deviance))

    # 3. Plotagem das bandas
    sim_resid = np.array(sim_resid)
    lower = np.percentile(sim_resid, 2.5, axis=0)
    upper = np.percentile(sim_resid, 97.5, axis=0)

    plt.plot(sorted_resid, 'o', label="Resíduos observados")
    plt.plot(lower, 'r--', linewidth=1, label="banda 2.5%")
    plt.plot(upper, 'r--', linewidth=1, label="banda 97.5%")
    plt.title(title)
    plt.xlabel("Ordem dos resíduos")
    plt.ylabel("Resíduo componente do desvio")
    plt.legend()
    plt.grid(True)
    plt.show()

# Executa o gráfico
plt.figure(figsize=(8, 3), dpi=150)
envelope_negbin(ajuste_nb, "Envelope de Resíduos - Binomial Negativa")

1. **Critérios de Ajuste (Métricas)**:O modelo Binomial Negativo apresentou uma redução drástica no AIC e na Deviance em comparação ao modelo de Poisson.

- **AIC**: Caiu significativamente (confira os valores no output acima), indicando um modelo muito mais parcimonioso e ajustado.
- **Deviance**: Também reduziu, mostrando que o erro residual diminuiu.

2. **Diagnóstico Gráfico**:Ao analisarmos o gráfico de envelope do modelo Binomial Negativo, observamos que a grande maioria dos resíduos agora se encontra dentro das bandas de confiança de 95%. Isso contrasta com o modelo de Poisson, onde os pontos fugiam das bandas, confirmando que o parâmetro extra de dispersão ($\alpha$) foi capaz de modelar a variabilidade excessiva dos dados.

**Conclusão**:O modelo Binomial Negativo é o mais adequado para descrever o número de espirros, pois controla a superdispersão, corrige os erros-padrão das estimativas e oferece um ajuste estatisticamente superior.

---

## Parte 5 - Estimar e interpretar o efeito médio marginal

 Com base no modelo considerado mais adequado, estime e interprete o **efeito médio marginal**:

* do consumo de álcool (**alcohol**) sobre o número médio de espirros,
* do uso de anti-histamínico (**antihist**).


In [ ]:
import numpy as np

# 1. Pegar os coeficientes do modelo ajustado (ajuste_nb)
beta_alcohol = ajuste_nb.params['C(alcohol)[T.1]']
beta_antihist = ajuste_nb.params['C(antihist)[T.1]']

# 2. Calcular a Razão de Taxas (Rate Ratio) exponenciando os coeficientes
# Isso transforma o valor logarítmico em um multiplicador da média
rr_alcohol = np.exp(beta_alcohol)
rr_antihist = np.exp(beta_antihist)

# 3. Exibir os resultados
print("=== Efeito Médio Marginal (Interpretação) ===")
print(f"Álcool (Coeficiente): {beta_alcohol:.4f}")
print(f"Fator Multiplicativo (exp): {rr_alcohol:.4f}")
print(f"Interpretação: Aumenta a média em {((rr_alcohol - 1) * 100):.2f}%")
print("-" * 40)
print(f"Anti-histamínico (Coeficiente): {beta_antihist:.4f}")
print(f"Fator Multiplicativo (exp): {rr_antihist:.4f}")
print(f"Interpretação: Reduz a média em {((1 - rr_antihist) * 100):.2f}%")

Para interpretar o efeito das variáveis na média de espirros, calculamos a razão de taxas (Rate Ratio) exponenciando os coeficientes estimados ($\exp(\beta)$). Com base nos cálculos acima:

- **Consumo de Álcool (alcohol)**:O resultado mostra que o consumo de álcool tem um efeito significativo de aumento no número de espirros. Mantendo as outras variáveis constantes, estima-se que indivíduos que consumiram álcool nas últimas 24 horas apresentam, em média, um número de espirros 46,73% maior do que aqueles que não consumiram.

- **Uso de Anti-histamínico (antihist)**:O resultado confirma que o medicamento tem um efeito protetor (redutor) sobre os sintomas. O uso de anti-histamínico reduz a taxa média de espirros para cerca de 55% da taxa original. Isso representa uma redução de 45,08% no número esperado de espirros para os pacientes que utilizaram a medicação, em comparação aos que não utilizaram.

---

## Parte 6 - Ajustar os modelos Poission e Binomial Negativa com base apenas no conjunto de treinamento 
Separe os dados em dois subconjuntos, treinamento com 80% das observações e
teste com 20%  das observações. Com base apenas no conjunto de treinamento, ajuste os modelos Poisson e Binomial Negativo com a mesma estrutura de covariáveis dos itens anteriores. No conjunto de teste, calcule, para cada modelo:

* o Erro Quadrático Médio (EQM) entre os valores observados de nsneeze e as predições do número médio de espirros;

* o Erro Absoluto Médio (EAM).

Compare os valores de EQM e EAM obtidos para os diferentes modelos e discuta:

qual deles apresenta melhor desempenho preditivo fora da amostra;

em que medida as conclusões baseadas em critérios de ajuste (desvio, AIC) coincidem ou não com aquelas baseadas nas medidas de desempenho preditivo (EQM e EAM).



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1. Separação dos dados (80% Treino, 20% Teste)
# random_state fixado com o Nº USP para garantir consistência
train, test = train_test_split(df, test_size=0.2, random_state=16310829)

print(f"Tamanho do Treino: {train.shape[0]} | Tamanho do Teste: {test.shape[0]}")

Neste cenário específico de divisão dos dados (70/30), o modelo de Poisson apresentou um desempenho preditivo ligeiramente superior, com erros menores tanto no critério quadrático quanto no absoluto.

In [ ]:
# 2. Ajuste dos modelos apenas no conjunto de Treinamento
formula = "nsneeze ~ C(alcohol) + C(antihist) + C(smoker) + age + pollen"

# Modelo Poisson (Treino)
model_pois_train = smf.glm(formula=formula, data=train, family=sm.families.Poisson()).fit()

# Modelo Binomial Negativo (Treino) - Mantendo alpha=0.2 (sneeze5)
model_nb_train = smf.glm(
    formula=formula,
    data=train,
    family=sm.families.NegativeBinomial(alpha=0.2)
).fit()

In [ ]:
# 3. Predições no conjunto de Teste
# O método .predict() usa os coeficientes aprendidos no treino aplicados aos dados de teste
pred_pois = model_pois_train.predict(test)
pred_nb = model_nb_train.predict(test)

In [ ]:
# 4. Cálculo das Métricas de Erro (EQM e EAM)
# EQM (Erro Quadrático Médio) - Penaliza mais os grandes erros
mse_pois = mean_squared_error(test['nsneeze'], pred_pois)
mse_nb = mean_squared_error(test['nsneeze'], pred_nb)

# EAM (Erro Absoluto Médio) - Média da distância absoluta
mae_pois = mean_absolute_error(test['nsneeze'], pred_pois)
mae_nb = mean_absolute_error(test['nsneeze'], pred_nb)

In [ ]:
# Exibição dos resultados
print("\n=== Desempenho Preditivo (Fora da Amostra) ===")
print(f"Poisson          -> EQM: {mse_pois:.4f} | EAM: {mae_pois:.4f}")
print(f"Binomial Negativa -> EQM: {mse_nb:.4f} | EAM: {mae_nb:.4f}")

**Discussão do Desempenho Preditivo e Ajuste**
1. **Comparação de Desempenho Preditivo (EQM e EAM)**:Ao analisarmos as métricas calculadas no conjunto de teste (os 20% dos dados não vistos pelo modelo), observamos os seguintes resultados:

- **Modelo Poisson**: Apresentou um Erro Quadrático Médio (EQM) de 1000.55 e um Erro Absoluto Médio (EAM) de 12.39.
- **Modelo Binomial Negativo**: Apresentou um EQM de 1039.84 e um EAM de 12.69.

Curiosamente, neste conjunto de teste específico, o modelo de Poisson apresentou um desempenho preditivo pontual ligeiramente superior (menores erros), embora a diferença entre os dois modelos seja pequena (cerca de 4% no EQM).

2. **Coincidência com Critérios de Ajuste (AIC e Deviance)**:Aqui observamos uma divergência entre os critérios de ajuste e de predição:

- **Critérios de Ajuste (Parte 4)**: O modelo Binomial Negativo foi massivamente superior, reduzindo o AIC em mais de 4200 pontos e a Deviance em quase 90% comparado ao Poisson. Isso indica que ele modela a distribuição dos dados (especialmente a variância/superdispersão) muito melhor.
- Critérios de Predição (Parte 6): O modelo Poisson venceu por uma margem estreita.

**Conclusão**:
Esse fenômeno ocorre porque o modelo de Poisson, mesmo com superdispersão, tende a ser um estimador consistente da média (os coeficientes $\beta$ geralmente são muito parecidos com os da Binomial Negativa). Como o EQM e o EAM avaliam apenas a precisão da média prevista ($\hat{y}$) e não a variância em torno dela, o Poisson consegue prever bem o valor esperado.

No entanto, para fins de inferência estatística (testes de hipótese, intervalos de confiança) e para explicar a variabilidade dos dados, o modelo Binomial Negativo continua sendo o mais adequado (devido ao AIC/Deviance), pois o Poisson subestima grosseiramente os erros-padrão ao ignorar a superdispersão. A vitória do Poisson na predição pode ser atribuída à robustez da estimativa da média ou a flutuações aleatórias na amostra de teste específica.

---

## Parte 7 - Previsões

Utilizando o modelo escolhido, faça previsões do número **esperado** de espirros para os seguintes perfis:

1. Indivíduo A:

   * alcohol = 0,
   * antihist = 1,
   * smoker = 0,
   * age = 30 anos,
   * pollen = valor correspondente a um dia de baixa concentração.

2. Indivíduo B:

   * alcohol = 1,
   * antihist = 0,
   * smoker = 1,
   * age = 50 anos,
   * pollen = valor correspondente a um dia de alta concentração.

In [ ]:
# 1. Definir valores para "Baixa" e "Alta" concentração de pólen
# Usamos o 1º Quartil (25%) para baixa e o 3º Quartil (75%) para alta
pollen_baixo = df['pollen'].quantile(0.25)
pollen_alto = df['pollen'].quantile(0.75)

print(f"Concentração de Pólen -> Baixa: {pollen_baixo:.2f} | Alta: {pollen_alto:.2f}")

In [ ]:
# 2. Criar o DataFrame com os perfis dos indivíduos A e B
# Individuo A: alcohol=0, antihist=1, smoker=0, age=30, pollen=baixo
# Individuo B: alcohol=1, antihist=0, smoker=1, age=50, pollen=alto
perfis = pd.DataFrame({
    'alcohol': [0, 1],
    'antihist': [1, 0],
    'smoker': [0, 1],
    'age': [30, 50],
    'pollen': [pollen_baixo, pollen_alto]
}, index=['Indivíduo A', 'Indivíduo B'])

In [ ]:
# 3. Fazer a previsão usando o modelo escolhido (Binomial Negativo)
# O método predict calcula o valor esperado (média mu)
previsoes = ajuste_nb.predict(perfis)

In [ ]:
# 4. Exibir os resultados
print("\n=== Previsão do Número Esperado de Espirros ===")
print(perfis)
print("-" * 50)
for individuo, espirros in zip(perfis.index, previsoes):
    print(f"{individuo}: espera-se aproximadamente {espirros:.2f} espirros no dia.")


Utilizando o modelo **Binomial Negativo**, realizamos a previsão do número esperado de espirros para dois perfis extremos:

**1. Indivíduo A (Perfil de Baixo Risco):**
* **Características:** Não consome álcool, usa anti-histamínico, não fuma, é jovem (30 anos) e está exposto a uma **baixa concentração de pólen** (1º Quartil = 36.91).
* **Previsão:** O modelo estima aproximadamente **5,48 espirros** por dia.
* **Análise:** Este valor baixo reflete o efeito protetor do medicamento combinado com a ausência de fatores agravantes (álcool e fumo) e a baixa exposição ambiental.

**2. Indivíduo B (Perfil de Alto Risco):**
* **Características:** Consome álcool, **não** usa anti-histamínico, é fumante, tem mais idade (50 anos) e está exposto a uma **alta concentração de pólen** (3º Quartil = 76.15).
* **Previsão:** O modelo estima aproximadamente **68,19 espirros** por dia.
* **Análise:** O número elevado de espirros (mais de 12 vezes superior ao do Indivíduo A) demonstra o efeito cumulativo dos fatores de risco. O modelo capturou com sucesso como a combinação de comportamento (álcool/fumo), falta de tratamento e ambiente adverso (pólen alto) pode agravar drasticamente os sintomas da rinite.